In [2]:
import pandas as pd
import numpy as np
import datetime
pd.options.mode.chained_assignment = None 

# A_chartevent. calculate the recording frequency for itemids in chartevent data (in 24-hour range) recording frequency: fraction of patients who record the certain itemid on all the hadm_ids

In [3]:
mimic_chart_MV = pd.read_csv("./data/mimic_chart_MV_only_numeric.csv")
mimic_chart_CV = pd.read_csv("./data/mimic_chart_CV_only_numeric.csv")

/Users/shutingcui/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (11,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
mimic_chart_MV.head()

,Unnamed: 0,Unnamed: 0.1,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped,expire_flag,dbsource
0,0,0,822284,10144,107460,283007,220194,2202-10-11 14:17:00,2202-10-11 14:20:00,14891.0,150.0,150.0,msec,0.0,0.0,NaN,NaN,1,metavision
1,1,137,822503,10144,107460,283007,220194,2202-10-11 18:09:00,2202-10-11 19:10:00,14891.0,150.0,150.0,msec,0.0,0.0,NaN,NaN,1,metavision
2,2,138,822421,10144,107460,283007,220194,2202-10-11 16:36:00,2202-10-11 16:38:00,14891.0,150.0,150.0,msec,0.0,0.0,NaN,NaN,1,metavision
3,3,139,2089196,18254,151644,274253,220194,2198-12-07 04:00:00,2198-12-07 04:11:00,17806.0,170.0,170.0,msec,0.0,0.0,NaN,NaN,0,metavision
4,4,140,2088915,18254,151644,274253,220194,2198-12-06 18:00:00,2198-12-06 18:32:00,14891.0,170.0,170.0,msec,0.0,0.0,NaN,NaN,0,metavision


In [5]:
mimic_chart_CV.head()

,Unnamed: 0,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped,expire_flag,dbsource
0,0,178984111,29130,106375,212954.0,53,2200-08-02 04:00:00,2200-08-02 04:00:00,21570,NaN,NaN,NaN,NaN,NaN,NaN,D/C'd,1,carevue
1,1,178984002,29130,106375,212954.0,53,2200-08-02 03:00:00,2200-08-02 05:33:00,19089,NaN,NaN,mmHg,NaN,NaN,NaN,NotStopd,1,carevue
2,1101,226498071,623,177331,292410.0,8555,2112-11-09 08:00:00,2112-11-09 08:41:00,15046,43.0,43.0,mmHg,NaN,NaN,NaN,NotStopd,1,carevue
3,1102,226498079,623,177331,292410.0,8555,2112-11-09 08:05:00,2112-11-09 08:41:00,15046,39.0,39.0,mmHg,NaN,NaN,NaN,NotStopd,1,carevue
4,1103,226498088,623,177331,292410.0,8555,2112-11-09 09:00:00,2112-11-09 09:07:00,15046,52.0,52.0,mmHg,NaN,NaN,NaN,NotStopd,1,carevue


* Step1 for A: 
    * compute the total number of hadm_ids in current dataset
    * groupby itemid -> get the hadm_id having the itemid recorded -> calculating the fraction for this itemid

* apply step1 on mimic_chart_MV

In [6]:
# chart_MV_partial is the simplify version of mimic_chart_MV in order to calculate the recording frequency of each itemid
chart_MV_partial = mimic_chart_MV[["hadm_id", "itemid"]]
chart_MV_itemid_list = chart_MV_partial["itemid"].unique()
chart_MV_gb_itemid = chart_MV_partial.groupby("itemid")



In [7]:
itemid_record_frequency_chart_MV = pd.DataFrame(np.zeros((1, len(chart_MV_itemid_list))), columns=list(chart_MV_itemid_list))
itemid_record_frequency_chart_MV

,220194,224643,224645,224646,224647,223962,223958,223960,223963,227565,...,226063,226457,226329,226183,226184,226092,226094,226096,226170,226272
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
chart_MV_total_num_patients = len(chart_MV_partial["hadm_id"].unique())

In [9]:

for i, itemid in enumerate(chart_MV_itemid_list):
    cur_itemid_recordings = chart_MV_gb_itemid.get_group(itemid)
    cur_itemid_hadmids_num = len(cur_itemid_recordings["hadm_id"].unique())
    itemid_record_frequency_chart_MV.loc[0][itemid] = cur_itemid_hadmids_num / chart_MV_total_num_patients

In [10]:
itemid_record_frequency_chart_MV = itemid_record_frequency_chart_MV.T
itemid_record_frequency_chart_MV.reset_index(inplace=True)
itemid_record_frequency_chart_MV.rename(columns={"index": "itemid", 0: "fraction of recorded hadmid"}, inplace=True)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(itemid_record_frequency_chart_MV)


,itemid,fraction of recorded hadmid
0,220194,0.002848
1,224643,0.012719
2,224645,0.003512
3,224646,0.003749
4,224647,0.002515
5,223962,0.106592
6,223958,0.092354
7,223960,0.088415
8,223963,0.010963
9,227565,0.313132


In [11]:
itemid_record_frequency_chart_MV.sort_values(by=["fraction of recorded hadmid"], ascending=False, inplace=True, ignore_index=True)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(itemid_record_frequency_chart_MV)

,itemid,fraction of recorded hadmid
0,220045,0.997105
1,220277,0.996630
2,220210,0.996203
3,220047,0.995017
4,220046,0.994969
5,223770,0.994827
6,223769,0.994163
7,224161,0.993925
8,224162,0.993878
9,220602,0.985857


* apply step1 on mimiv_chart_CV

In [12]:
chart_CV_partial = mimic_chart_CV[["hadm_id", "itemid"]]
chart_CV_gb_itemid = chart_CV_partial.groupby("itemid")
chart_CV_itemid_list = chart_CV_partial["itemid"].unique()
chart_CV_total_num_patients = len(chart_CV_partial["hadm_id"].unique())
itemid_record_frequency_chart_CV = pd.DataFrame(np.zeros((1, len(chart_CV_itemid_list))), columns=list(chart_CV_itemid_list))
for i, itemid in enumerate(chart_CV_itemid_list):
    cur_itemid_recordings = chart_CV_gb_itemid.get_group(itemid)
    cur_itemid_hadmids_num = len(cur_itemid_recordings["hadm_id"].unique())
    itemid_record_frequency_chart_CV.loc[0][itemid] = cur_itemid_hadmids_num / chart_CV_total_num_patients
itemid_record_frequency_chart_CV = itemid_record_frequency_chart_CV.T
itemid_record_frequency_chart_CV.reset_index(inplace=True)
itemid_record_frequency_chart_CV.rename(columns={"index": "itemid", 0: "fraction of recorded hadmid"}, inplace=True)
itemid_record_frequency_chart_CV.sort_values(by=["fraction of recorded hadmid"], ascending=False, inplace=True, ignore_index=True)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(itemid_record_frequency_chart_CV)

,itemid,fraction of recorded hadmid
0,211,0.996246
1,742,0.996207
2,646,0.995518
3,198,0.993679
4,618,0.993257
5,479,0.991380
6,829,0.983335
7,813,0.979926
8,811,0.979581
9,791,0.977742


# A_lab. calculate the recording frequency for itemids in lab data (in 24-hour range) recording frequency: fraction of patients who record the certain itemid on all the hadm_ids

* apply on mimic_lab_MV

In [42]:
mimic_lab_MV = pd.read_csv("./data/mimic_lab_MV_reg_pre.csv")
mimic_lab_CV = pd.read_csv("./data/mimic_lab_CV_reg_pre.csv")

In [43]:
lab_MV_partial = mimic_lab_MV[["hadm_id", "itemid"]]
lab_MV_itemid_list = lab_MV_partial["itemid"].unique()
lab_MV_gb_itemid = lab_MV_partial.groupby("itemid")

In [44]:
itemid_record_frequency_lab_MV = pd.DataFrame(np.zeros((1, len(lab_MV_itemid_list))), columns=list(lab_MV_itemid_list))
itemid_record_frequency_lab_MV

,50868,50882,50893,50902,50912,50931,50960,50970,50971,50983,...,51391,51368,51160,51442,51027,51030,51394,51415,51390,51395
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
lab_MV_total_num_patients = len(lab_MV_partial["hadm_id"].unique())

In [46]:
for i, itemid in enumerate(lab_MV_itemid_list):
    cur_itemid_recordings = lab_MV_gb_itemid.get_group(itemid)
    cur_itemid_hadmids_num = len(cur_itemid_recordings["hadm_id"].unique())
    itemid_record_frequency_lab_MV.loc[0][itemid] = cur_itemid_hadmids_num / lab_MV_total_num_patients
    
itemid_record_frequency_lab_MV = itemid_record_frequency_lab_MV.T
itemid_record_frequency_lab_MV.reset_index(inplace=True)
itemid_record_frequency_lab_MV.rename(columns={"index": "itemid", 0: "fraction of recorded hadmid"}, inplace=True)
itemid_record_frequency_lab_MV.sort_values(by=["fraction of recorded hadmid"], ascending=False, inplace=True, ignore_index=True)

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(itemid_record_frequency_lab_MV)

,itemid,fraction of recorded hadmid
0,50902,0.986982
1,50983,0.986462
2,50912,0.986272
3,50882,0.986130
4,51006,0.986083
5,50971,0.985704
6,51221,0.983243
7,50868,0.982911
8,50931,0.980166
9,51265,0.978083


* apply on mimic_lab_CV

In [47]:
lab_CV_partial = mimic_lab_CV[["hadm_id", "itemid"]]
lab_CV_itemid_list = lab_CV_partial["itemid"].unique()
lab_CV_gb_itemid = lab_CV_partial.groupby("itemid")

In [48]:
itemid_record_frequency_lab_CV = pd.DataFrame(np.zeros((1, len(lab_CV_itemid_list))), columns=list(lab_CV_itemid_list))
itemid_record_frequency_lab_CV

,51006,51010,51221,51222,51237,51248,51249,51250,51265,51274,...,51202,51366,51407,51368,51511,51454,50850,50989,51472,51308
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
lab_CV_total_num_patients = len(lab_CV_partial["hadm_id"].unique())
for i, itemid in enumerate(lab_CV_itemid_list):
    cur_itemid_recordings = lab_CV_gb_itemid.get_group(itemid)
    cur_itemid_hadmids_num = len(cur_itemid_recordings["hadm_id"].unique())
    itemid_record_frequency_lab_CV.loc[0][itemid] = cur_itemid_hadmids_num / lab_CV_total_num_patients
    
itemid_record_frequency_lab_CV = itemid_record_frequency_lab_CV.T
itemid_record_frequency_lab_CV.reset_index(inplace=True)
itemid_record_frequency_lab_CV.rename(columns={"index": "itemid", 0: "fraction of recorded hadmid"}, inplace=True)
itemid_record_frequency_lab_CV.sort_values(by=["fraction of recorded hadmid"], ascending=False, inplace=True, ignore_index=True)

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(itemid_record_frequency_lab_CV)

,itemid,fraction of recorded hadmid
0,50912,0.985176
1,51006,0.984525
2,51221,0.981843
3,50882,0.976749
4,50902,0.976404
5,51265,0.974795
6,50983,0.971539
7,51301,0.969317
8,51222,0.965372
9,51249,0.964644


## !!Note: After filtering those itemids with "fraction of recorded hadmid" less than 30%, we need to refilter itemids for lab data to make sure the itemids for the two eras are the same

In [53]:
selected_itemids_lab_MV = set(itemid_record_frequency_lab_MV[itemid_record_frequency_lab_MV["fraction of recorded hadmid"] >= 0.3]["itemid"].unique())
selected_itemids_lab_CV = set(itemid_record_frequency_lab_CV[itemid_record_frequency_lab_CV["fraction of recorded hadmid"] >= 0.3]["itemid"].unique())
print(len(selected_itemids_lab_MV))
print(len(selected_itemids_lab_CV))
final_itemids_lab_mv_cv = selected_itemids_lab_MV.intersection(selected_itemids_lab_CV)
print(len(final_itemids_lab_mv_cv))
# final_itemids_lab_mv_cv

36
41
36


# B. only select those itemids with "fraction of recorded hadmid" >= 0.3 both in dataframe "itemid_record_frequency_chart_MV" and "itemid_record_frequency_chart_CV"


In [13]:
# var "chart_MV_selected_itemid_list": itemids with "fraction of recorded hadmid" >= 0.3 
chart_MV_selected_itemid_list = itemid_record_frequency_chart_MV[itemid_record_frequency_chart_MV["fraction of recorded hadmid"] >= 0.3]["itemid"]
chart_CV_selected_itemid_list = itemid_record_frequency_chart_CV[itemid_record_frequency_chart_CV["fraction of recorded hadmid"] >= 0.3]["itemid"]
print(len(chart_MV_selected_itemid_list))
print(len(chart_CV_selected_itemid_list))

86
102


## Transforming temporal data into Ryan's figure's form (from row 1071 in draft_feature_matching.ipynb)

In [54]:
mimic_chart_MV_after_filter = mimic_chart_MV[mimic_chart_MV["itemid"].isin(chart_MV_selected_itemid_list)]
mimic_chart_CV_after_filter = mimic_chart_CV[mimic_chart_CV["itemid"].isin(chart_CV_selected_itemid_list)]
mimic_chart_MV_af_gb_hadmid = mimic_chart_MV_after_filter.groupby("hadm_id")
mimic_chart_CV_af_gb_hadmid = mimic_chart_CV_after_filter.groupby("hadm_id")

In [55]:
mimic_lab_MV_after_filter = mimic_lab_MV[mimic_lab_MV["itemid"].isin(final_itemids_lab_mv_cv)]
mimic_lab_CV_after_filter = mimic_lab_CV[mimic_lab_CV["itemid"].isin(final_itemids_lab_mv_cv)]
mimic_lab_MV_af_gb_hadmid = mimic_lab_MV_after_filter.groupby("hadm_id")
mimic_lab_CV_af_gb_hadmid = mimic_lab_CV_after_filter.groupby("hadm_id")

In [16]:
mimic_icu_adm = pd.read_csv("./data/mimic_icuadmittime.csv")

## function "Cur_Patient_24hr_itemids_value_chartevent" will get patient: "hadm_id" transformed itemids dataFrame for era "era" (for both lab data and chartevent data)

In [56]:
def Cur_Patient_24hr_itemids_value(dataset, hadm_id, era):
    cur_dataset_af_gb_hadmid = 0
    if dataset == "lab":
        if era == "MV":
            cur_dataset_af_gb_hadmid = mimic_lab_MV_af_gb_hadmid
        else:
            cur_dataset_af_gb_hadmid = mimic_lab_CV_af_gb_hadmid
    if dataset == "chartevent":
        if era == "MV":
            cur_dataset_af_gb_hadmid = mimic_chart_MV_af_gb_hadmid
        else:
            cur_dataset_af_gb_hadmid = mimic_chart_CV_af_gb_hadmid
        
#     if 
#         cur_adm_recordings = mimic_chart_MV_af_gb_hadmid.get_group(hadm_id)[["hadm_id", "itemid", "charttime", "value"]]
#     if era == "CV":
#         cur_adm_recordings = mimic_chart_CV_af_gb_hadmid.get_group(hadm_id)[["hadm_id", "itemid", "charttime", "value"]]
    cur_adm_recordings = cur_dataset_af_gb_hadmid.get_group(hadm_id)[["hadm_id", "itemid", "charttime", "value"]]
    
    cur_icu_adm_time = mimic_icu_adm[mimic_icu_adm["hadm_id"] == hadm_id]["intime"].unique()[0]
    icu_adm_timestamp = datetime.datetime.timestamp(datetime.datetime.strptime(cur_icu_adm_time,"%Y-%m-%d %H:%M:%S"))
    cur_adm_recordings[["charttimestamp"]] = cur_adm_recordings[["charttime"]].apply((lambda x: datetime.datetime.timestamp(datetime.datetime.strptime(x["charttime"],"%Y-%m-%d %H:%M:%S"))), axis=1)
    cur_adm_recordings[["charttime_interval_after_icu_adm"]] = (cur_adm_recordings[["charttimestamp"]] - icu_adm_timestamp) / 3600
    cur_adm_recordings[["charttime_interval_after_icu_adm"]] = cur_adm_recordings[["charttime_interval_after_icu_adm"]].astype(int) + 1 # "+1" because astype(int) will only keep the integer part of the float number
    # for example: charttime_interval_after_icu_adm == 0.2, actually this belongs to the first one-hour window, so the result should be 1, not 0 (if returned directly by astype(int))
    cur_adm_recordings.drop(columns=["charttime", "charttimestamp"], inplace=True) # columns: "hadm_id", "itemid", "charttime_interval_after_icu_adm", "value" are left 
    cur_adm_recordings = cur_adm_recordings[cur_adm_recordings["charttime_interval_after_icu_adm"] <= 24] # only keep the recordings in the first 24-hour window range
    cur_adm_recordings.sort_values(by="charttime_interval_after_icu_adm", inplace=True, ignore_index=True)
    cur_adm_itemids = cur_adm_recordings["itemid"].unique()
    cur_adm_gb_itemid = cur_adm_recordings.groupby("itemid")
    final_cur_adm_recordings = 0
    for j, itemid in enumerate(cur_adm_itemids):
        cur_item = cur_adm_gb_itemid.get_group(itemid)
        cur_item_first_value_one_window = cur_item.groupby("charttime_interval_after_icu_adm").first()
        if j == 0:
            final_cur_adm_recordings = cur_item_first_value_one_window
        else:
            final_cur_adm_recordings = pd.concat([final_cur_adm_recordings, cur_item_first_value_one_window])
    final_cur_adm_recordings.reset_index(inplace=True)
    final_cur_adm_recordings = final_cur_adm_recordings.pivot(index="charttime_interval_after_icu_adm", columns="itemid", values="value")
    
    return final_cur_adm_recordings



In [59]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(Cur_Patient_24hr_itemids_value("chartevent", 165660, "MV"))

itemid,220045,220046,220047,220179,220180,220181,220210,220224,220228,220235,220277,220292,220293,220339,220545,220546,220602,220615,220621,220635,220645,223751,223752,223761,223769,223770,223830,223834,223835,223873,223874,223875,223876,224161,224162,224684,224685,224687,224688,224689,224690,224695,224696,224697,224738,224828,225624,225625,225634,225668,225677,225698,226253,226871,226873,227073,227429,227442,227443,227457,227465,227466,227467,227565,227566
charttime_interval_after_icu_adm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,92.0,120.0,50.0,122.0,72.0,84.0,16.0,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.0,90.0,99.3,100.0,90.0,NaN,15.0,100.0,NaN,NaN,NaN,NaN,30.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,89.0,NaN,NaN,137.0,70.0,84.0,13.0,NaN,12.7,NaN,93.0,NaN,NaN,NaN,36.3,15.3,108.0,0.9,126.0,1.9,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,7.8,347.0,NaN,2.4,NaN,NaN,NaN,NaN,11.0,0.03,4.2,24.0,174.0,14.2,28.0,1.2,NaN,NaN
3,87.0,NaN,NaN,139.0,84.0,98.0,22.0,NaN,NaN,NaN,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,80.0,NaN,NaN,118.0,82.0,89.0,15.0,NaN,NaN,NaN,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,83.0,NaN,NaN,134.0,83.0,95.0,20.0,NaN,NaN,NaN,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,83.0,NaN,NaN,121.0,65.0,77.0,22.0,58.0,NaN,60.0,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.0,NaN,NaN,7.29,15.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,86.0,NaN,NaN,137.0,72.0,84.0,21.0,NaN,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,85.0,NaN,NaN,118.0,53.0,65.0,19.0,NaN,NaN,NaN,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,87.0,NaN,NaN,130.0,61.0,79.0,20.0,70.0,NaN,75.0,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.7,NaN,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
print(Cur_Patient_24hr_itemids_value("lab", 165660, "MV").shape[0])
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(Cur_Patient_24hr_itemids_value("lab", 165660, "MV"))


7


itemid,50802,50804,50813,50818,50820,50821,50868,50882,50893,50902,50910,50911,50912,50931,50960,50970,50971,50983,51006,51221,51222,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
charttime_interval_after_icu_adm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,11,24,7.8,108,347,6,0.9,126,1.9,2.4,4.2,139,19,36.3,12.7,1.2,29.5,34.9,85,174,14.2,28.0,14.8,4.29,15.3
5,0,30,1.0,60,7.29,58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,32,0.7,75,7.22,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.5,NaN,1.3,NaN,NaN,NaN,NaN,14.5,61.7,NaN,NaN,NaN
11,1,29,0.9,50,7.36,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,11,27,NaN,106,NaN,NaN,1.0,101,1.9,NaN,3.9,140,20,32.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.3,NaN,NaN,NaN
22,NaN,NaN,NaN,NaN,NaN,NaN,9,28,7.9,109,NaN,NaN,1.0,105,1.9,2.1,4.0,142,22,32.7,11.5,1.4,29.4,35.2,84,146,16.2,49.5,14.7,3.91,10.9


In [62]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(Cur_Patient_24hr_itemids_value("lab", 124930, "CV"))

itemid,50868,50882,50893,50902,50910,50911,50912,50931,50960,50970,50971,50983,51006,51221,51222,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
charttime_interval_after_icu_adm,,,,,,,,,,,,,,,,,,,,,,,,,
6,14,24,9.4,103,NaN,4,0.8,147,1.9,2.9,4.1,137,15,41.3,14.1,1.2,30.1,34.3,88,272,13.4,53.1,13.7,4.69,13.0
11,NaN,NaN,NaN,NaN,61,NotDone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,58,NotDone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.2,NaN,NaN,NaN
24,15,23,8.8,105,58,NotDone,0.7,122,2.2,2.5,3.9,139,14,40.7,13.8,1.2,29.8,34.0,88,288,13.8,55.8,13.7,4.65,14.5


In [63]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(Cur_Patient_24hr_itemids_value("chartevent", 124930, "CV"))

itemid,87,198,211,455,456,470,479,618,646,677,678,742,781,784,786,787,788,791,811,813,814,815,821,824,825,827,828,829,833,837,861,1127,1162,1286,1522,1523,1525,1529,1530,1532,1533,1534,1535,1536,1542,5815,5817,5819,5820,8441,8549,8551,8553,8554
charttime_interval_after_icu_adm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,17.0,15.0,68.0,168.0,97.0,2.0,NaN,26.0,97.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,76.0,120.0,210.0,30.0,100.0
2,NaN,15.0,63.0,172.0,92.0,NaN,NaN,23.0,95.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,66.0,120.0,210.0,30.0,100.0
3,NaN,15.0,76.0,135.0,88.0,2.0,NaN,18.0,98.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,71.0,120.0,210.0,30.0,100.0
4,NaN,15.0,60.0,154.0,85.0,NaN,NaN,20.0,98.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,61.0,120.0,210.0,30.0,100.0
5,NaN,15.0,74.0,136.0,82.0,NaN,NaN,17.0,99.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,62.0,120.0,200.0,30.0,100.0
6,NaN,15.0,64.0,150.0,91.0,2.0,NaN,20.0,97.0,NaN,NaN,1.0,15.0,NaN,9.4,24.0,103.0,0.8,147.0,41.299999,14.1,1.2,1.9,13.4,53.1,2.9,272.0,4.1,4.69,137.0,13.0,13.0,15.0,13.4,9.4,103.0,0.8,147.0,1.2,1.9,53.1,2.9,4.1,137.0,13.0,55.0,90.0,8.0,93.0,71.0,120.0,200.0,30.0,100.0
7,NaN,15.0,66.0,140.0,85.0,2.0,NaN,21.0,97.0,35.944401,96.699997,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,69.0,120.0,200.0,30.0,100.0
8,NaN,15.0,95.0,162.0,94.0,2.0,NaN,28.0,98.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,78.0,120.0,200.0,30.0,100.0
9,NaN,15.0,77.0,143.0,82.0,2.0,NaN,19.0,95.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,69.0,120.0,200.0,30.0,100.0


# !!! The block below (In[69] shows that there is no patient in lab_MV that has 24-hour itemid recordings )

In [69]:
final_lab_MV_hadmid_list = mimic_lab_MV_after_filter["hadm_id"].unique()
for patient in final_lab_MV_hadmid_list:
    cur_patient_transformed_itemid_recording = Cur_Patient_24hr_itemids_value("lab", patient, "MV")
    if cur_patient_transformed_itemid_recording.shape[0] == 24:
        with pd.option_context("display.max_rows", None, "display.max_columns", None):
            display(cur_patient_transformed_itemid_recording)


In [70]:
final_lab_CV_hadmid_list = mimic_lab_CV_after_filter["hadm_id"].unique()
for patient in final_lab_CV_hadmid_list:
    cur_patient_transformed_itemid_recording = Cur_Patient_24hr_itemids_value("lab", patient, "CV")
    if cur_patient_transformed_itemid_recording.shape[0] == 24:
        with pd.option_context("display.max_rows", None, "display.max_columns", None):
            display(cur_patient_transformed_itemid_recording)


## !! function "Imputation" used to impute NaN values in lab/chart event data and make them have the final shape with 24 rows and columns are the recorded itemid (both numeric or categorical)

In [78]:
fill_limit_lab = pd.read_csv("./data/fill_limit_lab.csv")
fill_limit_lab.head()

,itemid,recommend_fill_limit_CV,recommend_fill_limit_MV
0,50820,4.0,5.0
1,50800,4.0,5.0
2,50821,4.0,4.0
3,50804,4.0,4.0
4,50818,4.0,4.0


In [79]:
# To check whether all the selected itemids for lab_mv_cv are included in the recommend_fill_limit_value: yes
print(len(final_itemids_lab_mv_cv))
check_common_itemid_lab = final_itemids_lab_mv_cv.intersection(fill_limit_lab["itemid"].unique())
print(len(check_common_itemid_lab))

36
36


In [80]:
fill_limit_chart_MV = pd.read_csv("./data/fill_limit_chart_MV.csv")
fill_limit_chart_MV.head()

,itemid,recommend_fill_limit_MV
0,220045,1.0
1,220210,1.0
2,220277,1.0
3,220181,1.0
4,220179,1.0


In [81]:
fill_limit_chart_CV = pd.read_csv("./data/fill_limit_chart_CV.csv")
fill_limit_chart_CV.head()

,itemid,recommend_fill_limit_CV
0,211,1.0
1,742,1.0
2,618,1.0
3,646,1.0
4,51,1.0


# Imputation Steps:
## A.linear interpolation with recommend_fill_limit_value and limit_area="inside"

## B.Then apply backward and forward fill with recommend_fill_limit_value and limit_area="outside"

In [127]:
# "recording_range" == 24 (hrs) currently
def Imputation(dataset, era, org_itemid_records, recording_range):
    # First get 
    fill_limit_list = 0
    if dataset == "lab":
        fill_limit_list = fill_limit_lab
    else:
        if era == "MV":
            fill_limit_list = fill_limit_chart_MV
        else:
            fill_limit_list = fill_limit_chart_CV
    imputed_itemid_recordings = 0
    # Firstly extend current pd DataFrame with 24 rows
    if org_itemid_records.shape[0] != 24:
        current_index = set(org_itemid_records.index)
        expected_index = set(range(1, recording_range+1))
        missed_index = expected_index - current_index
        # "df_interpolate_rows" is the rows being interpolated on org_itemid_records with all NaN values
        df_interpolate_rows = pd.DataFrame(np.nan, index=missed_index, columns=org_itemid_records.columns)
        imputed_itemid_recordings = pd.concat([org_itemid_records, df_interpolate_rows]).sort_index(ascending=True)
        
    else:
        imputed_itemid_recordings = org_itemid_records
    cur_hadm_itemids = org_itemid_records.columns
    for itemid in cur_hadm_itemids:
        cur_itemid_fill_limit = 0
        if era == "CV":
            cur_itemid_fill_limit = round(fill_limit_list[fill_limit_list["itemid"]==itemid]["recommend_fill_limit_CV"].values[0])
        else: # era == "MV"
            cur_itemid_fill_limit = round(fill_limit_list[fill_limit_list["itemid"]==itemid]["recommend_fill_limit_MV"].values[0])
        if dataset == "lab":
            imputed_itemid_recordings[itemid] = pd.to_numeric(imputed_itemid_recordings[itemid], errors='coerce')
        cur_itemid_recordings = imputed_itemid_recordings[[itemid]]   
        imputed_cur_itemid_recordings = cur_itemid_recordings.interpolate(method="linear", axis=0, limit=cur_itemid_fill_limit, limit_area="inside")
        imputed_cur_itemid_recordings = imputed_cur_itemid_recordings.interpolate(axis=0, limit=cur_itemid_fill_limit, limit_direction="both", limit_area="outside")
        imputed_itemid_recordings[[itemid]] = imputed_cur_itemid_recordings
        
    
    return imputed_itemid_recordings

## Below code blocks is for checking whether the function "Imputation" works
* "chartevent", 165660, "MV"
* "lab", 165660, "MV"
* "chartevent", 124930, "CV"
* "lab", 124930, "CV"

* **LAB_CV is correct**

In [128]:
org_itemid_recordings = Cur_Patient_24hr_itemids_value("lab", 124930, "CV")
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(org_itemid_recordings)

imputed_itemid_recordings = Imputation("lab", "CV", org_itemid_recordings, 24)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(imputed_itemid_recordings)

itemid,50868,50882,50893,50902,50910,50911,50912,50931,50960,50970,50971,50983,51006,51221,51222,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
charttime_interval_after_icu_adm,,,,,,,,,,,,,,,,,,,,,,,,,
6,14,24,9.4,103,NaN,4,0.8,147,1.9,2.9,4.1,137,15,41.3,14.1,1.2,30.1,34.3,88,272,13.4,53.1,13.7,4.69,13.0
11,NaN,NaN,NaN,NaN,61,NotDone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,58,NotDone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.2,NaN,NaN,NaN
24,15,23,8.8,105,58,NotDone,0.7,122,2.2,2.5,3.9,139,14,40.7,13.8,1.2,29.8,34.0,88,288,13.8,55.8,13.7,4.65,14.5


itemid,50868,50882,50893,50902,50910,50911,50912,50931,50960,50970,50971,50983,51006,51221,51222,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
1,14.000000,24.000000,9.400000,103.000000,NaN,4.0,0.800000,147.000000,1.900000,2.900000,4.100000,137.000000,15.000000,41.300000,14.100000,1.2,30.100000,34.300000,88.0,272.000000,13.400000,53.100000,13.7,4.690000,13.000000
2,14.000000,24.000000,9.400000,103.000000,61.000000,4.0,0.800000,147.000000,1.900000,2.900000,4.100000,137.000000,15.000000,41.300000,14.100000,1.2,30.100000,34.300000,88.0,272.000000,13.400000,53.100000,13.7,4.690000,13.000000
3,14.000000,24.000000,9.400000,103.000000,61.000000,4.0,0.800000,147.000000,1.900000,2.900000,4.100000,137.000000,15.000000,41.300000,14.100000,1.2,30.100000,34.300000,88.0,272.000000,13.400000,53.100000,13.7,4.690000,13.000000
4,14.000000,24.000000,9.400000,103.000000,61.000000,4.0,0.800000,147.000000,1.900000,2.900000,4.100000,137.000000,15.000000,41.300000,14.100000,1.2,30.100000,34.300000,88.0,272.000000,13.400000,53.100000,13.7,4.690000,13.000000
5,14.000000,24.000000,9.400000,103.000000,61.000000,4.0,0.800000,147.000000,1.900000,2.900000,4.100000,137.000000,15.000000,41.300000,14.100000,1.2,30.100000,34.300000,88.0,272.000000,13.400000,53.100000,13.7,4.690000,13.000000
6,14.000000,24.000000,9.400000,103.000000,61.000000,4.0,0.800000,147.000000,1.900000,2.900000,4.100000,137.000000,15.000000,41.300000,14.100000,1.2,30.100000,34.300000,88.0,272.000000,13.400000,53.100000,13.7,4.690000,13.000000
7,14.055556,23.944444,9.366667,103.111111,61.000000,4.0,0.794444,145.611111,1.916667,2.877778,4.088889,137.111111,14.944444,41.266667,14.083333,1.2,30.083333,34.283333,88.0,272.888889,13.422222,54.680000,13.7,4.687778,13.083333
8,14.111111,23.888889,9.333333,103.222222,61.000000,4.0,0.788889,144.222222,1.933333,2.855556,4.077778,137.222222,14.888889,41.233333,14.066667,1.2,30.066667,34.266667,88.0,273.777778,13.444444,56.260000,13.7,4.685556,13.166667
9,14.166667,23.833333,9.300000,103.333333,61.000000,4.0,0.783333,142.833333,1.950000,2.833333,4.066667,137.333333,14.833333,41.200000,14.050000,1.2,30.050000,34.250000,88.0,274.666667,13.466667,57.840000,13.7,4.683333,13.250000
10,14.222222,23.777778,9.266667,103.444444,61.000000,4.0,0.777778,141.444444,1.966667,2.811111,4.055556,137.444444,14.777778,41.166667,14.033333,1.2,30.033333,34.233333,88.0,275.555556,13.488889,59.420000,13.7,4.681111,13.333333


* **chart_CV is correct**

In [129]:
org_itemid_recordings = Cur_Patient_24hr_itemids_value("chartevent", 124930, "CV")
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(org_itemid_recordings)

imputed_itemid_recordings = Imputation("chart", "CV", org_itemid_recordings, 24)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(imputed_itemid_recordings)

itemid,87,198,211,455,456,470,479,618,646,677,678,742,781,784,786,787,788,791,811,813,814,815,821,824,825,827,828,829,833,837,861,1127,1162,1286,1522,1523,1525,1529,1530,1532,1533,1534,1535,1536,1542,5815,5817,5819,5820,8441,8549,8551,8553,8554
charttime_interval_after_icu_adm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,17.0,15.0,68.0,168.0,97.0,2.0,NaN,26.0,97.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,76.0,120.0,210.0,30.0,100.0
2,NaN,15.0,63.0,172.0,92.0,NaN,NaN,23.0,95.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,66.0,120.0,210.0,30.0,100.0
3,NaN,15.0,76.0,135.0,88.0,2.0,NaN,18.0,98.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,71.0,120.0,210.0,30.0,100.0
4,NaN,15.0,60.0,154.0,85.0,NaN,NaN,20.0,98.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,61.0,120.0,210.0,30.0,100.0
5,NaN,15.0,74.0,136.0,82.0,NaN,NaN,17.0,99.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,62.0,120.0,200.0,30.0,100.0
6,NaN,15.0,64.0,150.0,91.0,2.0,NaN,20.0,97.0,NaN,NaN,1.0,15.0,NaN,9.4,24.0,103.0,0.8,147.0,41.299999,14.1,1.2,1.9,13.4,53.1,2.9,272.0,4.1,4.69,137.0,13.0,13.0,15.0,13.4,9.4,103.0,0.8,147.0,1.2,1.9,53.1,2.9,4.1,137.0,13.0,55.0,90.0,8.0,93.0,71.0,120.0,200.0,30.0,100.0
7,NaN,15.0,66.0,140.0,85.0,2.0,NaN,21.0,97.0,35.944401,96.699997,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,69.0,120.0,200.0,30.0,100.0
8,NaN,15.0,95.0,162.0,94.0,2.0,NaN,28.0,98.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,78.0,120.0,200.0,30.0,100.0
9,NaN,15.0,77.0,143.0,82.0,2.0,NaN,19.0,95.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,90.0,8.0,93.0,69.0,120.0,200.0,30.0,100.0


itemid,87,198,211,455,456,470,479,618,646,677,678,742,781,784,786,787,788,791,811,813,814,815,821,824,825,827,828,829,833,837,861,1127,1162,1286,1522,1523,1525,1529,1530,1532,1533,1534,1535,1536,1542,5815,5817,5819,5820,8441,8549,8551,8553,8554
charttime_interval_after_icu_adm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,17.0,15.0,68.0,168.0,97.0,2.0,NaN,26.0,97.0,NaN,NaN,1.0,15.000000,NaN,9.400000,24.000000,103.000000,0.800000,147.00,41.299999,14.100000,1.2,1.900000,13.400000,53.100000,2.900000,272.000000,4.100000,4.690000,137.000000,13.000000,13.000000,15.000000,13.400000,9.400000,103.000000,0.800000,147.00,1.2,1.900000,53.100000,2.900000,4.100000,137.000000,13.000000,55.0,90.0,8.0,93.0,76.0,120.0,210.0,30.0,100.0
2,17.0,15.0,63.0,172.0,92.0,2.0,NaN,23.0,95.0,NaN,NaN,1.0,15.000000,61.000000,9.400000,24.000000,103.000000,0.800000,147.00,41.299999,14.100000,1.2,1.900000,13.400000,53.100000,2.900000,272.000000,4.100000,4.690000,137.000000,13.000000,13.000000,15.000000,13.400000,9.400000,103.000000,0.800000,147.00,1.2,1.900000,53.100000,2.900000,4.100000,137.000000,13.000000,55.0,90.0,8.0,93.0,66.0,120.0,210.0,30.0,100.0
3,17.0,15.0,76.0,135.0,88.0,2.0,NaN,18.0,98.0,35.944401,96.699997,1.0,15.000000,61.000000,9.400000,24.000000,103.000000,0.800000,147.00,41.299999,14.100000,1.2,1.900000,13.400000,53.100000,2.900000,272.000000,4.100000,4.690000,137.000000,13.000000,13.000000,15.000000,13.400000,9.400000,103.000000,0.800000,147.00,1.2,1.900000,53.100000,2.900000,4.100000,137.000000,13.000000,55.0,90.0,8.0,93.0,71.0,120.0,210.0,30.0,100.0
4,17.0,15.0,60.0,154.0,85.0,2.0,NaN,20.0,98.0,35.944401,96.699997,1.0,15.000000,61.000000,9.400000,24.000000,103.000000,0.800000,147.00,41.299999,14.100000,1.2,1.900000,13.400000,53.100000,2.900000,272.000000,4.100000,4.690000,137.000000,13.000000,13.000000,15.000000,13.400000,9.400000,103.000000,0.800000,147.00,1.2,1.900000,53.100000,2.900000,4.100000,137.000000,13.000000,55.0,90.0,8.0,93.0,61.0,120.0,210.0,30.0,100.0
5,17.0,15.0,74.0,136.0,82.0,2.0,NaN,17.0,99.0,35.944401,96.699997,1.0,15.000000,61.000000,9.400000,24.000000,103.000000,0.800000,147.00,41.299999,14.100000,1.2,1.900000,13.400000,53.100000,2.900000,272.000000,4.100000,4.690000,137.000000,13.000000,13.000000,15.000000,13.400000,9.400000,103.000000,0.800000,147.00,1.2,1.900000,53.100000,2.900000,4.100000,137.000000,13.000000,55.0,90.0,8.0,93.0,62.0,120.0,200.0,30.0,100.0
6,17.0,15.0,64.0,150.0,91.0,2.0,NaN,20.0,97.0,35.944401,96.699997,1.0,15.000000,61.000000,9.400000,24.000000,103.000000,0.800000,147.00,41.299999,14.100000,1.2,1.900000,13.400000,53.100000,2.900000,272.000000,4.100000,4.690000,137.000000,13.000000,13.000000,15.000000,13.400000,9.400000,103.000000,0.800000,147.00,1.2,1.900000,53.100000,2.900000,4.100000,137.000000,13.000000,55.0,90.0,8.0,93.0,71.0,120.0,200.0,30.0,100.0
7,17.0,15.0,66.0,140.0,85.0,2.0,NaN,21.0,97.0,35.944401,96.699997,1.0,14.944444,61.000000,9.366666,23.944444,103.111111,0.794444,143.25,41.266666,14.083334,1.2,1.916667,13.422222,54.680000,2.877778,272.888889,4.088889,4.687778,137.111111,13.083333,13.083333,14.944444,13.422222,9.366666,103.111111,0.794444,143.25,1.2,1.916667,54.680000,2.877778,4.088889,137.111111,13.083333,55.0,90.0,8.0,93.0,69.0,120.0,200.0,30.0,100.0
8,17.0,15.0,95.0,162.0,94.0,2.0,NaN,28.0,98.0,35.966640,96.739998,1.0,14.888889,61.000000,9.333333,23.888889,103.222222,0.788889,139.50,41.233333,14.066667,1.2,1.933333,13.444444,56.260000,2.855556,273.777778,4.077778,4.685556,137.222222,13.166667,13.166667,14.888889,13.444444,9.333333,103.222222,0.788889,139.50,1.2,1.933333,56.260000,2.855556,4.077778,137.222222,13.166667,55.0,90.0,8.0,93.0,78.0,120.0,200.0,30.0,100.0
9,17.0,15.0,77.0,143.0,82.0,2.0,NaN,19.0,95.0,35.988880,96.779999,1.0,14.833333,61.000000,9.300000,23.833333,103.333333,0.783333,135.75,41.199999,14.050000,1.2,1.950000,13.466667,57.840000,2.833333,274.666667,4.066667,4.683333,137.333333,13.250000,13.250000,14.833333,13.466667,9.300000,103.333333,0.783333,135.75,

* **LAB_MV is correct**

In [130]:
org_itemid_recordings = Cur_Patient_24hr_itemids_value("lab", 165660, "MV")
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(org_itemid_recordings)

imputed_itemid_recordings = Imputation("lab", "MV", org_itemid_recordings, 24)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(imputed_itemid_recordings)

itemid,50802,50804,50813,50818,50820,50821,50868,50882,50893,50902,50910,50911,50912,50931,50960,50970,50971,50983,51006,51221,51222,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
charttime_interval_after_icu_adm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,11,24,7.8,108,347,6,0.9,126,1.9,2.4,4.2,139,19,36.3,12.7,1.2,29.5,34.9,85,174,14.2,28.0,14.8,4.29,15.3
5,0,30,1.0,60,7.29,58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,32,0.7,75,7.22,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.5,NaN,1.3,NaN,NaN,NaN,NaN,14.5,61.7,NaN,NaN,NaN
11,1,29,0.9,50,7.36,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,11,27,NaN,106,NaN,NaN,1.0,101,1.9,NaN,3.9,140,20,32.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.3,NaN,NaN,NaN
22,NaN,NaN,NaN,NaN,NaN,NaN,9,28,7.9,109,NaN,NaN,1.0,105,1.9,2.1,4.0,142,22,32.7,11.5,1.4,29.4,35.2,84,146,16.2,49.5,14.7,3.91,10.9


itemid,50802,50804,50813,50818,50820,50821,50868,50882,50893,50902,50910,50911,50912,50931,50960,50970,50971,50983,51006,51221,51222,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
1,0.000000,30.000000,1.000000,60.000000,7.290000,58.000000,11.000000,24.000000,7.800000,108.000000,347.0,6.0,0.900000,126.000000,1.9,2.400000,4.200000,139.000000,19.000000,36.300000,12.700000,1.200000,29.500000,34.900000,85.000000,174.000000,14.200000,28.000000,14.800000,4.290000,15.300000
2,0.000000,30.000000,1.000000,60.000000,7.290000,58.000000,11.000000,24.200000,7.804762,107.866667,347.0,6.0,0.906667,124.333333,1.9,2.385714,4.180000,139.066667,19.066667,36.322222,12.642857,1.211111,29.495238,34.914286,84.952381,172.666667,14.233333,31.744444,14.795238,4.271905,15.090476
3,0.000000,30.000000,1.000000,60.000000,7.290000,58.000000,11.000000,24.400000,7.809524,107.733333,347.0,6.0,0.913333,122.666667,1.9,2.371429,4.160000,139.133333,19.133333,36.344444,12.585714,1.222222,29.490476,34.928571,84.904762,171.333333,14.266667,35.488889,14.790476,4.253810,14.880952
4,0.000000,30.000000,1.000000,60.000000,7.290000,58.000000,11.000000,24.600000,7.814286,107.600000,347.0,6.0,0.920000,121.000000,1.9,2.357143,4.140000,139.200000,19.200000,36.366667,12.528571,1.233333,29.485714,34.942857,84.857143,170.000000,14.300000,39.233333,14.785714,4.235714,14.671429
5,0.000000,30.000000,1.000000,60.000000,7.290000,58.000000,11.000000,24.800000,7.819048,107.466667,347.0,6.0,0.926667,119.333333,1.9,2.342857,4.120000,139.266667,19.266667,36.388889,12.471429,1.244444,29.480952,34.957143,84.809524,168.666667,14.333333,42.977778,14.780952,4.217619,14.461905
6,0.000000,30.666667,0.900000,65.000000,7.266667,62.000000,11.000000,25.000000,7.823810,107.333333,347.0,6.0,0.933333,117.666667,1.9,2.328571,4.100000,139.333333,19.333333,36.411111,12.414286,1.255556,29.476190,34.971429,84.761905,167.333333,14.366667,46.722222,14.776190,4.199524,14.252381
7,0.000000,31.333333,0.800000,70.000000,7.243333,66.000000,11.000000,25.200000,7.828571,107.200000,347.0,6.0,0.940000,116.000000,1.9,2.314286,4.080000,139.400000,19.400000,36.433333,12.357143,1.266667,29.471429,34.985714,84.714286,166.000000,14.400000,50.466667,14.771429,4.181429,14.042857
8,0.000000,32.000000,0.700000,75.000000,7.220000,70.000000,11.000000,25.400000,7.833333,107.066667,347.0,6.0,0.946667,114.333333,1.9,2.300000,4.060000,139.466667,19.466667,36.455556,12.300000,1.277778,29.466667,35.000000,84.666667,164.666667,14.433333,54.211111,14.766667,4.163333,13.833333
9,0.333333,31.000000,0.766667,66.666667,7.266667,74.666667,11.000000,25.600000,7.838095,106.933333,347.0,6.0,0.953333,112.666667,1.9,2.285714,4.040000,139.533333,19.533333,36.477778,12.242857,1.288889,29.461905,35.014286,84.619048,163.333333,14.466667,57.955556,14.761905,4.145238,13.623810
10,0.666667,30.000000,0.833333,58.333333,7.313333,79.333333,11.000000,25.800000,7.842857,106.800000,347.0,6.0,0.960000,111.000000,1.9,2.271429,4.020000,139.600000,19.600000,36.500000,12.185714,1.300000,29.457143,35.028571,84.571429,162.000000,14.500000,61.700000,14.757143,4.127143,13.414286


* **chart_MV is correct**

In [131]:
org_itemid_recordings = Cur_Patient_24hr_itemids_value("chartevent", 165660, "MV")
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(org_itemid_recordings)

imputed_itemid_recordings = Imputation("chart", "MV", org_itemid_recordings, 24)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(imputed_itemid_recordings)

itemid,220045,220046,220047,220179,220180,220181,220210,220224,220228,220235,220277,220292,220293,220339,220545,220546,220602,220615,220621,220635,220645,223751,223752,223761,223769,223770,223830,223834,223835,223873,223874,223875,223876,224161,224162,224684,224685,224687,224688,224689,224690,224695,224696,224697,224738,224828,225624,225625,225634,225668,225677,225698,226253,226871,226873,227073,227429,227442,227443,227457,227465,227466,227467,227565,227566
charttime_interval_after_icu_adm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,92.0,120.0,50.0,122.0,72.0,84.0,16.0,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.0,90.0,99.3,100.0,90.0,NaN,15.0,100.0,NaN,NaN,NaN,NaN,30.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,89.0,NaN,NaN,137.0,70.0,84.0,13.0,NaN,12.7,NaN,93.0,NaN,NaN,NaN,36.3,15.3,108.0,0.9,126.0,1.9,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,7.8,347.0,NaN,2.4,NaN,NaN,NaN,NaN,11.0,0.03,4.2,24.0,174.0,14.2,28.0,1.2,NaN,NaN
3,87.0,NaN,NaN,139.0,84.0,98.0,22.0,NaN,NaN,NaN,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,80.0,NaN,NaN,118.0,82.0,89.0,15.0,NaN,NaN,NaN,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,83.0,NaN,NaN,134.0,83.0,95.0,20.0,NaN,NaN,NaN,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,83.0,NaN,NaN,121.0,65.0,77.0,22.0,58.0,NaN,60.0,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.0,NaN,NaN,7.29,15.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,86.0,NaN,NaN,137.0,72.0,84.0,21.0,NaN,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,85.0,NaN,NaN,118.0,53.0,65.0,19.0,NaN,NaN,NaN,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,87.0,NaN,NaN,130.0,61.0,79.0,20.0,70.0,NaN,75.0,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.7,NaN,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


itemid,220045,220046,220047,220179,220180,220181,220210,220224,220228,220235,220277,220292,220293,220339,220545,220546,220602,220615,220621,220635,220645,223751,223752,223761,223769,223770,223830,223834,223835,223873,223874,223875,223876,224161,224162,224684,224685,224687,224688,224689,224690,224695,224696,224697,224738,224828,225624,225625,225634,225668,225677,225698,226253,226871,226873,227073,227429,227442,227443,227457,227465,227466,227467,227565,227566
charttime_interval_after_icu_adm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,92.0,120.0,50.0,122.0,72.0,84.0,16.0,NaN,12.700000,NaN,85.0,NaN,NaN,NaN,36.300000,15.300000,108.000000,0.900000,126.000000,1.9,139.000000,160.0,90.000000,99.300,100.0,90.0,7.290000,15.0,100.0,NaN,NaN,NaN,NaN,30.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.000000,7.800000,347.0,1.000000,2.400000,NaN,85.0,NaN,NaN,11.000000,0.03,4.200000,24.000000,174.000000,14.200000,28.000000,1.200000,NaN,NaN
2,89.0,120.0,50.0,137.0,70.0,84.0,13.0,58.000000,12.700000,60.000000,93.0,NaN,NaN,NaN,36.300000,15.300000,108.000000,0.900000,126.000000,1.9,139.000000,160.0,89.230769,98.840,100.0,90.0,7.290000,15.0,100.0,NaN,NaN,NaN,NaN,30.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,19.000000,7.800000,347.0,1.000000,2.400000,30.000000,85.0,NaN,NaN,11.000000,0.03,4.200000,24.000000,174.000000,14.200000,28.000000,1.200000,1200.0,1600.0
3,87.0,120.0,50.0,139.0,84.0,98.0,22.0,58.000000,12.642857,60.000000,95.0,NaN,NaN,NaN,36.322222,15.090476,107.866667,0.906667,124.333333,1.9,139.066667,160.0,88.461538,98.380,100.0,90.0,7.290000,15.0,100.0,NaN,NaN,NaN,NaN,30.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,19.066667,7.804762,347.0,1.000000,2.385714,30.000000,85.0,NaN,NaN,11.000000,0.03,4.180000,24.200000,172.666667,14.233333,31.744444,1.211111,1200.0,1600.0
4,80.0,120.0,50.0,118.0,82.0,89.0,15.0,58.000000,12.585714,60.000000,96.0,NaN,NaN,NaN,36.344444,14.880952,107.733333,0.913333,122.666667,1.9,139.133333,160.0,87.692308,97.920,100.0,90.0,7.290000,15.0,100.0,NaN,NaN,NaN,NaN,30.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,19.133333,7.809524,347.0,1.000000,2.371429,30.000000,85.0,NaN,NaN,11.000000,0.03,4.160000,24.400000,171.333333,14.266667,35.488889,1.222222,1200.0,1600.0
5,83.0,120.0,50.0,134.0,83.0,95.0,20.0,58.000000,12.528571,60.000000,91.0,NaN,NaN,NaN,36.366667,14.671429,107.600000,0.920000,121.000000,1.9,139.200000,160.0,86.923077,97.460,100.0,90.0,7.290000,15.0,100.0,NaN,NaN,NaN,NaN,30.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.000000,NaN,NaN,0.000000,19.200000,7.814286,347.0,1.000000,2.357143,30.000000,85.0,NaN,NaN,11.000000,0.03,4.140000,24.600000,170.000000,14.300000,39.233333,1.233333,1200.0,1600.0
6,83.0,120.0,50.0,121.0,65.0,77.0,22.0,58.000000,12.471429,60.000000,95.0,4.0,21.0,5.0,36.388889,14.461905,107.466667,0.926667,119.333333,1.9,139.266667,160.0,86.153846,97.000,100.0,90.0,7.290000,15.0,100.0,40.00,1.400,10.0,20.00,30.0,8.0,500.0,534.000000,11.000,20.0,0.000000,20.000000,24.000000,16.000000,10.000000,0.85,0.000000,19.266667,7.819048,347.0,1.000000,2.342857,30.000000,85.0,2.50,1.0,11.000000,0.03,4.120000,24.800000,168.666667,14.333333,42.977778,1.244444,1200.0,1600.0
7,86.0,120.0,50.0,137.0,72.0,84.0,21.0,62.000000,12.414286,65.000000,93.0,4.0,21.0,5.0,36.411111,14.252381,107.333333,0.933333,117.666667,1.9,139.333333,160.0,85.384615,97.075,100.0,90.0,7.266667,15.0,100.0,40.00,1.400,10.0,20.00,30.0,8.0,500.0,534.000000,11.000,20.0,0.000000,20.000000,24.000000,16.000000,10.000000,0.85,0.000000,19.333333,7.823810,347.0,0.900000,2.328571,30.666667,85.0,2.50,1.0,11.000000,0.03,4.100000,25.000000,167.333333,14.366667,46.722222,1.255556,1200.0,1600.0
8,85.0,120.0,50.0,118.0,53.0,65.0,19.0,66.000000,12.357143,70.000000,91.0,4.0,21.0,5.0,36.433333,14.042857,107.200000,0.940000,116.000000,1.9,139.400000,160.0,84.615385,97.150,100.0,90.0,7.243333,15.0,100.0,40.00,1.400,10.0,20.00,30.0,8.0,500.0,534.000000,11.000,20.0,0.000000,20.000000,24.000000,16.000000,10.000000,0.85